In [1]:
import os
import pandas as pd

In [2]:
cohort_path = "/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/"
tasks = ['hospital_mortality','sepsis','LOS_7','readmission_30','hyperkalemia_lab_mild_label','hyperkalemia_lab_moderate_label','hyperkalemia_lab_severe_label','hyperkalemia_lab_abnormal_label','hypoglycemia_lab_mild_label','hypoglycemia_lab_moderate_label','hypoglycemia_lab_severe_label','hypoglycemia_lab_abnormal_label','neutropenia_lab_mild_label','neutropenia_lab_moderate_label','neutropenia_lab_severe_label','hyponatremia_lab_mild_label','hyponatremia_lab_moderate_label','hyponatremia_lab_severe_label','hyponatremia_lab_abnormal_label','aki_lab_aki1_label','aki_lab_aki2_label','aki_lab_aki3_label','aki_lab_abnormal_label','anemia_lab_mild_label','anemia_lab_moderate_label','anemia_lab_severe_label','anemia_lab_abnormal_label','thrombocytopenia_lab_mild_label','thrombocytopenia_lab_moderate_label','thrombocytopenia_lab_severe_label','thrombocytopenia_lab_abnormal_label']
random_state=44
def read_file(filename, columns=None, **kwargs):
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [3]:
cohort = read_file(
    os.path.join(
        cohort_path,
        "cohort/cohort_split_no_nb.parquet"
    ),
    engine='pyarrow'
)
cohort = cohort.query('pediatric_age_group!="term neonatal"')

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort/cohort_split_no_nb.parquet


In [4]:
seed = 345

In [5]:
cohort['constrain'] = 0

In [6]:
ped_cohort = cohort.query("adult_at_admission==0")

In [7]:
tr = ped_cohort.query("fold_id=='0'")
v = ped_cohort.query("fold_id=='val'")

In [9]:
for percent in list(range(5,100,5)):
    tr_df = tr.sample(frac=percent/100.0, random_state=seed)
    tr_pids = tr_df['person_id'].values
    v_df = v.sample(frac=percent/100.0, random_state=seed)
    v_pids = v_df['person_id'].values
    perc_df = cohort.copy()
    perc_df['constrain'].loc[perc_df['person_id'].isin(tr_pids)] = 1
    perc_df['constrain'].loc[perc_df['person_id'].isin(v_pids)] = 1

    perc_df.to_parquet(
        f"/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort/cohort_split_no_nb_constrain_{percent}.parquet",
        engine="pyarrow",
    )

/tmp/ipykernel_2246009/599771163.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_df['constrain'].loc[perc_df['person_id'].isin(tr_pids)] = 1
/tmp/ipykernel_2246009/599771163.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_df['constrain'].loc[perc_df['person_id'].isin(v_pids)] = 1
